In [3]:
import numpy as np
import pyvista as pv

def binary_mask_to_surface_stl(
    mask: np.ndarray,                  # 0/1 array shaped (nx, ny, nz)
    spacing=(1.0, 1.0, 1.0),           # (dx, dy, dz)
    origin=(0.0, 0.0, 0.0),
    target_edge=None,                  # desired ~edge length
    max_reduction=0.85,
    taubin_iter=30,
    pass_band=0.1,
    volume_tol_pct=1.0,
    out_stl_path="surface.stl",
):
    # --- pad one voxel of background ---
    padded = np.pad(mask, ((1,1),(1,1),(1,1)), mode="constant", constant_values=0)

    # --- ImageData grid (safer than UniformGrid on older PyVista) ---
    nx, ny, nz = padded.shape
    grid = pv.ImageData()
    grid.dimensions = (nx, ny, nz)     # number of POINTS per axis
    grid.spacing = spacing
    grid.origin = origin
    grid.point_data["scalars"] = padded.ravel(order="F")

    # --- extract surface (FlyingEdges via .contour) ---
    surf = grid.contour(isosurfaces=[0.5]).clean()

    # --- compute reduction from target_edge ---
    area = float(surf.area)
    tri_now = int(surf.n_cells)
    reduction = 0.0
    if target_edge is not None and tri_now > 0 and area > 0:
        tri_target = 4.0 * area / (np.sqrt(3.0) * (target_edge ** 2))
        reduction = 1.0 - min(1.0, tri_target / tri_now)
        reduction = float(np.clip(reduction, 0.0, max_reduction))

    # --- topology-preserving decimation ---
    if reduction > 0.0:
        surf = surf.decimate_pro(
            reduction=reduction,
            preserve_topology=True,
            feature_angle=45.0,
            split_angle=75.0,
            splitting=True,
        )

    # --- manifold & watertight check (pre-smooth) ---
    edges = surf.extract_feature_edges(
        boundary_edges=True, non_manifold_edges=True,
        feature_edges=False, manifold_edges=False
    )
    if edges.n_cells > 0:
        raise RuntimeError("Surface not closed after decimation. Use smaller reduction.")

    vol_before = float(surf.volume)

    # --- non-shrinking smoothing ---
    surf = surf.smooth_taubin(n_iter=taubin_iter, pass_band=pass_band, edge_angle=120.0)

    # --- re-check + volume drift guard ---
    edges = surf.extract_feature_edges(
        boundary_edges=True, non_manifold_edges=True,
        feature_edges=False, manifold_edges=False
    )
    if edges.n_cells > 0:
        raise RuntimeError("Surface lost closed/manifold property after smoothing. "
                           "Reduce taubin_iter or increase pass_band.")
    vol_after = float(surf.volume)
    drift_pct = (abs(vol_after - vol_before) / max(1e-12, vol_before)) * 100.0
    if drift_pct > volume_tol_pct:
        raise RuntimeError(f"Volume drift {drift_pct:.2f}% > tolerance ({volume_tol_pct}%).")

    # --- STL export ---
    surf = surf.triangulate()
    surf.compute_normals(inplace=True)
    surf.save(out_stl_path, binary=True)
    return surf


mask = np.load("data/sphere.npy")

mesh = binary_mask_to_surface_stl(
    mask,
    spacing=(1.0, 1.0, 1.0),
    origin=(0.0, 0.0, 0.0),
    target_edge=1.0,
    taubin_iter=25,
    pass_band=0.1,
    volume_tol_pct=10.0,
    out_stl_path="data/test.stl",
)

pv.set_jupyter_backend("trame")
plotter = pv.Plotter()
plotter.add_mesh(mesh, show_edges=True, smooth_shading=True)
plotter.add_axes()
plotter.show_grid()

if not pv.OFF_SCREEN:
    plotter.show()  




Widget(value='<iframe src="http://localhost:46091/index.html?ui=P_0x7c213a478350_2&reconnect=auto" class="pyvi…